In [2]:
n = 6
init_tellers = 10 # A0
cost_salary = 10 #CS
cost_retire = 30 #CR
cost_hire = 10 #CH
retire_max = 10 #RM
max_labor = 160 #MH

hours_created = [4800 6000 6500 5000 4500 3500] #HrsReq (work created in each month)
max_hrs_left = [800 700 600 500 400 0]; #Lt (max hours left over at the end of each month)

In [15]:
using JuMP, Cbc

m = Model()

@variable(m, x[1:n] >= 0) # of hours done in month t
@variable(m, y[1:n] >= 0) # of tellers working in month t
@variable(m, w[1:n] >= 0) # of leftover hours in month t
@variable(m, h[1:n] >= 0) # of hired workers in month t
@variable(m, r[1:n] >= 0) # of retired workers in month t

@objective(m, Min, sum(cost_salary * y) + sum(cost_hire * h) + sum(cost_retire * r))

@constraint(m, leftover_balance[t in 2:n], w[t] == w[t-1] + hours_created[t] - x[t])
@constraint(m, initial_leftover, w[1] == 0)
@constraint(m, max_leftoverp[t in 1:n], w[t] <= max_hrs_left[t])
@constraint(m, terminal_leftover, w[n] == 0)
@constraint(m, max_tot_retire, sum(r) <= retire_max)
@constraint(m, work_hours_limit[t in 1:n], x[t] <= max_labor * y[t])
@constraint(m, tellers_balance[t in 2:n], y[t] == y[t-1] + h[t] - r[t])
@constraint(m, initial_teller, y[1] == init_tellers)

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

println("Each month, hire and retire the following:")
for i in 1:n
    println("Month ", i)
    println("Hire: ", value(h[i]))
    println("Retire: ", value(r[i]))
    println("Working: ", value(y[i]))
    println("Carry-over hrs: ", value(w[i]))
end

Each month, hire and retire the following:
Month 1
Hire: 0.0
Retire: 0.0
Working: 10.0
Carry-over hrs: 0.0
Month 2
Hire: 27.1875
Retire: 0.0
Working: 37.1875
Carry-over hrs: 50.0
Month 3
Hire: 0.0
Retire: 0.0
Working: 37.1875
Carry-over hrs: 600.0
Month 4
Hire: 0.0
Retire: 5.3125
Working: 31.875
Carry-over hrs: 500.0
Month 5
Hire: 0.0
Retire: 0.0
Working: 31.875
Carry-over hrs: 0.0
Month 6
Hire: 0.0
Retire: 0.0
Working: 31.875
Carry-over hrs: 0.0
Presolve 11 (-15) rows, 19 (-11) columns and 37 (-25) elements
Perturbing problem by 0.001% of 30 - largest nonzero change 0.00019270511 ( 0.0018774106%) - largest zero change 5.9934174e-05
0  Obj 1781.5255 Primal inf 32.999992 (9)
8  Obj 2231.2801
Optimal - objective value 2231.25
After Postsolve, objective 2231.25, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2231.25 - 8 iterations time 0.012, Presolve 0.01


In [15]:
using JuMP, Cbc

m = Model()

@variable(m, x[1:n] >= 0) # of hired employees in period t
@variable(m, y[1:n] >= 0) # of retired employees in period t
@variable(m, z[0:n] >= 0) # of current employees in period t
@variable(m, s[0:n] >= 0) # of carryover hours in period t
@variable(m, a[1:n] >= 0) # of individual's working hours in period t

@objective(m, Min, sum(cost_salary * z) + sum(cost_retire * y) + sum(cost_hire * x))

@constraint(m, curr_work[t in 1:n], z[t] == z[t-1] + x[t] - y[t])
@constraint(m, carryover[t in 1:6], s[t] == s[t-1] + (hours_created[t] - (a[t] * z[t])))
#@constraint(m, carryover[t in 1:6], s[t] == (hours_created[t] - (a[t] * z[t])))
@constraint(m, start_worker, z[0] == init_tellers)
@constraint(m, start_carryover, s[0] == 0)
@constraint(m, max_retire, sum(y) <= retire_max)
@constraint(m, max_indv_labor[t in 1:6], a[t] <= max_labor)
@constraint(m, max_carryover[t in 1:6], s[t] <= max_hrs_left[t])
@constraint(m, tot_hrs_meet, sum(a[t] * z[t] + s[t] for t in 1:6)== sum(hours_created))

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

LoadError: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.EqualTo{Float64}` constraint is not supported by the model.